# 모델 구현 방식 비교

### 제시된 방법
- 방법 1: **직접 nn.Module을 상속하여 구현** &rarr; 더 많은 제어 가능성
- 방법 2: **nn.Sequential을 사용하여 구현** &rarr; 코드가 더 간단하고 유지보수가 용이
- 방법 3: **리스트로 묶어 제시하는 방식** &rarr; 간결함과 유연성을 결합
<br/><br/>

### 1. 직접 nn.Module을 상속하여 구현

**장점**
- 더 많은 제어 가능성: 각 레이어와 연산을 세밀하게 조작할 수 있습니다.
- 디버깅 용이: 각 단계별로 출력을 확인하고 문제를 찾기 쉽습니다.

**단점**
- 코드가 더 길고 복잡해질 수 있음: 특히 레이어가 많을 경우 코드가 길어지고 복잡해질 수 있습니다.
- 반복적인 코드: 레이어를 추가하거나 변경할 때 코드의 여러 부분을 수정해야 할 수 있습니다.

In [3]:
### CycleGAN Generator - Method 1
import torch
import torch.nn as nn

class ResidualBlock(nn.Module):
    def __init__(self, in_features):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_features, in_features, kernel_size=3, padding=1)
        self.in1 = nn.InstanceNorm2d(in_features)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(in_features, in_features, kernel_size=3, padding=1)
        self.in2 = nn.InstanceNorm2d(in_features)

    def forward(self, x):
        residual = x
        x = self.conv1(x)
        x = self.in1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.in2(x)
        x += residual
        return x

class Generator(nn.Module):
    def __init__(self, input_nc, output_nc, n_residual_blocks=9):
        super(Generator, self).__init__()

        self.conv1 = nn.Conv2d(input_nc, 64, kernel_size=7, padding=3)
        self.in1 = nn.InstanceNorm2d(64)
        self.relu = nn.ReLU(inplace=True)

        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1)
        self.in2 = nn.InstanceNorm2d(128)

        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, stride=2, padding=1)
        self.in3 = nn.InstanceNorm2d(256)

        self.residual_blocks = nn.Sequential(
            *[ResidualBlock(256) for _ in range(n_residual_blocks)]
        )

        self.deconv1 = nn.ConvTranspose2d(256, 128, kernel_size=3, stride=2, padding=1, output_padding=1)
        self.in4 = nn.InstanceNorm2d(128)

        self.deconv2 = nn.ConvTranspose2d(128, 64, kernel_size=3, stride=2, padding=1, output_padding=1)
        self.in5 = nn.InstanceNorm2d(64)

        self.conv4 = nn.Conv2d(64, output_nc, kernel_size=7, padding=3)
        self.tanh = nn.Tanh()

    def forward(self, x):
        x = self.relu(self.in1(self.conv1(x)))
        x = self.relu(self.in2(self.conv2(x)))
        x = self.relu(self.in3(self.conv3(x)))
        x = self.residual_blocks(x)
        x = self.relu(self.in4(self.deconv1(x)))
        x = self.relu(self.in5(self.deconv2(x)))
        x = self.tanh(self.conv4(x))
        return x

# Generator 모델 생성 및 요약
input_nc = 3  # J, H, K 입력 채널
output_nc = 1  # ch1 출력 채널
generator = Generator(input_nc=input_nc, output_nc=output_nc)

<br/>

### 2. nn.Sequential을 사용하여 구현

**장점**
- 코드가 간결하고 이해하기 쉬움: 레이어들을 순서대로 정의하면 되므로 코드가 더 간결합니다.
- 유지보수 용이: 레이어를 추가하거나 제거할 때 코드 수정이 간편합니다.

**단점**
- 제어가 제한적임: nn.Sequential을 사용하면 각 레이어 사이에 추가 연산을 삽입하기 어려워집니다.
- 복잡한 네트워크 구성 시 어려움: 조건문이나 반복문을 사용하여 네트워크를 구성할 때 제한적입니다.

In [4]:
import torch
import torch.nn as nn

class ResidualBlock(nn.Module):
    def __init__(self, in_features):
        super(ResidualBlock, self).__init__()
        self.block = nn.Sequential(
            nn.Conv2d(in_features, in_features, kernel_size=3, padding=1),
            nn.InstanceNorm2d(in_features),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_features, in_features, kernel_size=3, padding=1),
            nn.InstanceNorm2d(in_features)
        )

    def forward(self, x):
        return x + self.block(x)

class Generator(nn.Module):
    def __init__(self, input_nc, output_nc, n_residual_blocks=9):
        super(Generator, self).__init__()

        self.model = nn.Sequential(
            nn.Conv2d(input_nc, 64, kernel_size=7, padding=3),
            nn.InstanceNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1),
            nn.InstanceNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 256, kernel_size=3, stride=2, padding=1),
            nn.InstanceNorm2d(256),
            nn.ReLU(inplace=True),
            *[ResidualBlock(256) for _ in range(n_residual_blocks)],
            nn.ConvTranspose2d(256, 128, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.InstanceNorm2d(128),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(128, 64, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.InstanceNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, output_nc, kernel_size=7, padding=3),
            nn.Tanh()
        )

    def forward(self, x):
        return self.model(x)

# Generator 모델 생성 및 요약
input_nc = 3  # J, H, K 입력 채널
output_nc = 1  # ch1 출력 채널
generator = Generator(input_nc=input_nc, output_nc=output_nc)

<br/>

### 3. 리스트로 묶어 제시하는 방식

**장점**
- nn.Sequential의 간결함과 nn.Module의 유연성을 결합: 코드가 간결하면서도 유연한 제어가 가능합니다.
- 반복적인 구조를 쉽게 정의: 여러 레이어를 반복적으로 정의할 때 유용합니다.

**단점**
- 초기 학습 곡선: 이 방식에 익숙해지기까지 시간이 걸릴 수 있습니다.
- 디버깅 어려움: 리스트 내의 레이어들 사이에서 버그를 찾기 어려울 수 있습니다.

In [5]:
### CycleGAN Generator - Method 3
import torch
import torch.nn as nn

class ResidualBlock(nn.Module):
    def __init__(self, in_features):
        super(ResidualBlock, self).__init__()
        self.block = nn.Sequential(
            nn.Conv2d(in_features, in_features, kernel_size=3, padding=1),
            nn.InstanceNorm2d(in_features),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_features, in_features, kernel_size=3, padding=1),
            nn.InstanceNorm2d(in_features)
        )

    def forward(self, x):
        return x + self.block(x)

class Generator(nn.Module):
    def __init__(self, input_nc, output_nc, n_residual_blocks=9):
        super(Generator, self).__init__()

        layers = []
        layers.append(nn.Conv2d(input_nc, 64, kernel_size=7, padding=3))
        layers.append(nn.InstanceNorm2d(64))
        layers.append(nn.ReLU(inplace=True))
        
        layers.append(nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1))
        layers.append(nn.InstanceNorm2d(128))
        layers.append(nn.ReLU(inplace=True))
        
        layers.append(nn.Conv2d(128, 256, kernel_size=3, stride=2, padding=1))
        layers.append(nn.InstanceNorm2d(256))
        layers.append(nn.ReLU(inplace=True))
        
        for _ in range(n_residual_blocks):
            layers.append(ResidualBlock(256))
        
        layers.append(nn.ConvTranspose2d(256, 128, kernel_size=3, stride=2, padding=1, output_padding=1))
        layers.append(nn.InstanceNorm2d(128))
        layers.append(nn.ReLU(inplace=True))
        
        layers.append(nn.ConvTranspose2d(128, 64, kernel_size=3, stride=2, padding=1, output_padding=1))
        layers.append(nn.InstanceNorm2d(64))
        layers.append(nn.ReLU(inplace=True))
        
        layers.append(nn.Conv2d(64, output_nc, kernel_size=7, padding=3))
        layers.append(nn.Tanh())

        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

# Generator 모델 생성 및 요약
input_nc = 3  # J, H, K 입력 채널
output_nc = 1  # ch1 출력 채널
generator = Generator(input_nc=input_nc, output_nc=output_nc)